# Make-o-Matic Gesture Recognition

## Part 3: Machine Learning

2017 by Thomas Lidy, TU Wien

### Requirements

Python 2.7

pip install -r requirements.txt

Tested on OS: Ubuntu 16.04.3 LTS

In [29]:
import numpy as np
import pandas as pd
import json
import time # for time measuring
import datetime # for time printing

from scipy import stats
from scipy.signal import resample
# Power spectral density using a periodogram
from scipy.signal import periodogram

from collections import Counter # for majority vote
from collections import OrderedDict # for color palette

# Machine Learning
from sklearn import preprocessing, svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [30]:
def str_to_int(string):
    '''cut away first character and convert to int - used to convert Gesture IDs like "G01" to 1'''
    return int(string[1:])

In [31]:
def timestr(seconds):
    ''' returns HH:MM:ss formatted time string for given seconds
    (seconds can be a float with milliseconds included, but only the integer part will be used)
    :return: string
    '''
    return str(datetime.timedelta(seconds=int(seconds)))

## Read Meta-Data

In [32]:
# main data

# original input
#csv_file = 'data/EXPORT_09042017173622.csv'

# preprocessed input
csv_file = 'data/EXPORT_09042017173622_preprocessed.csv'


# json files to translate gestures, parcours into long text
#gestures_file = 'data/gestures.json' # this is the file edited manually by us to conform to json
gestures_file = 'data/gestures.json.orig' # this is the file edited manually by us to conform to json
parcours_file = 'data/parcours.json'
mutations_file = 'data/mutations.json'

files = (gestures_file, parcours_file, mutations_file)
dataframes = []

# NOTE THAT THESE JSON FILES ARE NOT JSON CONFORM
# each line is a json string on its own, so we need to process the json line by line and combine THEN into a list

In [33]:
def get_oid(oid_dict):
    # get from the original representation {u'$oid': u'589c8ed31337b5ab1e1be121'} just the oid
    return oid_dict['$oid']

In [34]:
# get meta-files with descriptions of gestures, parcours and mutations
for filename in files:
    with open(filename) as f:
        lines = [line.rstrip('\n') for line in f]   # .decode("utf-8")

    lines = [json.loads(line) for line in lines]
    
    # convert list of json lines into Dataframe
    df = pd.DataFrame.from_dict(lines)
    
    # convert long $oid to short
    df['_id'] = df['_id'].apply(get_oid)
    
    # set the real id
    df.set_index('id', inplace=True)
    
    # convert index (ID) from string like 'G01' to int
    df.index = df.index.map(str_to_int)
    
    dataframes.append(df)

In [35]:
(gestures_df, parcours_df, mutations_df) = tuple(dataframes)

In [36]:
gestures_df

,_id,isGarbage,isNesture,name,slug
1,58a23a22d826756404709446,NaN,NaN,Single Rotation klein rechtsrum,rssr
2,58a23a22d826756404709447,NaN,NaN,Single Rotation klein linksrum,rssl
3,58a23a22d826756404709448,NaN,NaN,Oszillierende Rotation klein rechtsrum,rosr
4,58a23a22d826756404709449,NaN,NaN,Oszillierende Rotation klein linksrum,rosl
5,58a23a22d82675640470944a,NaN,NaN,Single Rotation groß rechtsrum,rsbr
6,58a23a22d82675640470944b,NaN,NaN,Single Rotation groß linksrum,rsbl
7,58a23a22d82675640470944c,NaN,NaN,Oszillierende Rotation groß rechtsrum,robr
8,58a23a22d82675640470944d,NaN,NaN,Oszillierende Rotation groß linksrum,robl
9,58a23a22d82675640470944e,NaN,NaN,Kontinuierliche Rotation groß rechtsrum,rcbr
10,58a23a22d82675640470944f,NaN,NaN,Kontinuierliche Rotation groß linksrum,rcbl


In [37]:
# "positive" gestures to recognize (not nestures)
gestures_pos = gestures_df[gestures_df['isNesture'] != True].index.tolist()
gestures_pos

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [38]:
# "negative" gestures (nestures)
gestures_neg = gestures_df[gestures_df['isNesture'] == True].index.tolist()
nestures = gestures_neg # synonym
gestures_neg

[14, 15, 16, 17, 18]

#### Define handy function shortcut

In [39]:
def gesture_name(gesture_id):
    if gesture_id is None: return None
    return gestures_df.loc[gesture_id,'name']

## Read Experiment Data

In [40]:
# Experiment Data
data = pd.read_csv(csv_file)
data.head(10)

,Trainset,Experiment,Subject,TimeStamp,RFID,GRASP_A,GRASP_B,GRASP_C,AX,AY,AZ,EX,EY,EZ,Parcours,Parcours_Step,Mutation,Host,Host/Spot,Gesture
0,_TRAINSET14022017094616,1,Andreas,0,000000000000,781,8,797,0.06,-0.02,-0.10,216.8125,9.0625,-81.9375,101,1,151,8,NaN,15
1,_TRAINSET14022017094616,1,Andreas,29001,000000000000,782,0,799,0.09,-0.04,-0.11,217.0625,9.0625,-81.9375,101,1,151,8,NaN,15
2,_TRAINSET14022017094616,1,Andreas,46136,000000000000,782,6,798,0.12,-0.09,0.09,217.4375,9.1250,-81.8750,101,1,151,8,NaN,15
3,_TRAINSET14022017094616,1,Andreas,74902,000000000000,784,7,798,0.08,-0.08,0.03,217.6250,9.1250,-81.8125,101,1,151,8,NaN,15
4,_TRAINSET14022017094616,1,Andreas,97663,000000000000,781,0,798,0.07,-0.09,0.04,217.9375,9.1875,-81.7500,101,1,151,8,NaN,15
5,_TRAINSET14022017094616,1,Andreas,116448,000000000000,784,4,800,0.12,-0.06,-0.03,218.3125,9.2500,-81.7500,101,1,151,8,NaN,15
6,_TRAINSET14022017094616,1,Andreas,148753,000000000000,783,0,798,0.21,-0.04,0.03,218.5000,9.3125,-81.7500,101,1,151,8,NaN,15
7,_TRAINSET14022017094616,1,Andreas,167422,000000000000,784,2,798,0.18,-0.10,-0.08,218.6875,9.3750,-81.7500,101,1,151,8,NaN,15
8,_TRAINSET14022017094616,1,Andreas,187481,000000000000,782,4,799,0.15,-0.18,-0.03,219.0000,9.4375,-81.7500,101,1,151,8,NaN,15
9,_TRAINSET14022017094616,1,Andreas,213733,000000000000,784,13,799,0.15,-0.18,-0.17,219.1250,9.4375,-81.7500,101,1,151,8,NaN,15


In [41]:
# get the TimeStamp Diffs 
# (TimeStamps are reset after each Parcours, so we have to do it groupwise by Parcours)
group_by = ('Subject','Experiment','Trainset','Parcours')
timestamp_deltas = data.groupby(group_by)['TimeStamp'].diff()
#timestamp_deltas

In [42]:
timestamp_deltas.describe()

count    7.108210e+05
mean     2.586989e+04
std      3.509427e+04
min      1.930000e+02
25%      1.757400e+04
50%      2.400500e+04
75%      2.863500e+04
max      5.743871e+06
dtype: float64

In [43]:
# Note: Not sure why there is a value of 5,743,871 as the max timestamp delta

### Time Stamp delta and Sample Rate of Signal

In [44]:
time_delta = timestamp_deltas.median()
print "Time stamp deltas are on median", time_delta, "micro-seconds"

Time stamp deltas are on median 24005.0 micro-seconds


In [45]:
time_delta_sec = time_delta / 1000000  # microsec to sec
time_delta_sec

0.024005

In [46]:
sampling_rate = 1 /  time_delta_sec   
print "Signal sampling rate is on avg.", sampling_rate, "Hz"

Signal sampling rate is on avg. 41.6579879192 Hz


### Iterate through the data

In [47]:
# Note: Pandas group_by can be used instead of a for loop, to loop over groups of objects in the data, 
# e.g. all data in a parcours

In [48]:
# see gestures per Parcours
group_by = ('Subject','Experiment','Trainset','Parcours')
data.groupby(group_by)['Gesture'].unique()

Subject  Experiment  Trainset                 Parcours
Alfred   2           _TRAINSET14022017144824  101              [15, 1, 17]
                     _TRAINSET14022017144923  102              [15, 2, 17]
                     _TRAINSET14022017145122  103               [15, 1, 2]
                     _TRAINSET14022017145237  104               [15, 2, 1]
                     _TRAINSET14022017145434  107              [15, 1, 17]
                     _TRAINSET14022017145514  108              [15, 2, 17]
                     _TRAINSET14022017145629  109               [15, 1, 2]
                     _TRAINSET14022017145751  110               [15, 2, 1]
                     _TRAINSET14022017145913  113              [15, 1, 17]
                     _TRAINSET14022017145944  114              [15, 2, 17]
                     _TRAINSET14022017150026  115               [15, 1, 2]
                     _TRAINSET14022017150110  116               [15, 2, 1]
                     _TRAINSET14022017150614 

In [49]:
# see gestures per Parcours Step
group_by = ('Subject','Experiment','Trainset','Parcours','Parcours_Step')
data.groupby(group_by)['Gesture'].unique()

Subject  Experiment  Trainset                 Parcours  Parcours_Step
Alfred   2           _TRAINSET14022017144824  101       1                [15]
                                                        2                 [1]
                                                        3                [17]
                                                        4                 [1]
                                                        5                [17]
                                                        6                 [1]
                                                        7                [17]
                                                        8                 [1]
                                                        9                [17]
                                                        10                [1]
                     _TRAINSET14022017144923  102       1                [15]
                                                        2               

## Replace Nestures

choose whether to replace, i.e. merge nestures by their closest (following) gestures

In [50]:
replace_nestures = False

In [51]:
group_by = ('Subject','Experiment','Trainset','Parcours','Parcours_Step','Mutation','Gesture')
group_df = data.groupby(group_by)
print "Originally", len(group_df), "individual gesture blocks"

Originally 5169 individual gesture blocks


In [52]:
data.groupby(group_by).count().head(20)

TimeStamp  \
Subject Experiment Trainset                Parcours Parcours_Step Mutation Gesture              
Alfred  2          _TRAINSET14022017144824 101      1             151      15             142   
                                                    2             101      1               72   
                                                    3             152      17             102   
                                                    4             101      1               57   
                                                    5             152      17              77   
                                                    6             101      1               49   
                                                    7             152      17              66   
                                                    8             101      1               46   
                                                    9             152      17              65   
                                                    10            101      1               44   
                   _TRAINSET14022017144923 102      1             151      15              88   
                                                    2             102      2               54   
                                                    3             153      17              74   
                                                    4             102      2               37   
                                                    5             153      17              61   
                                                    6             102      2               41   
                                                    7             153      17              50   
                                                    8             102      2               35   
                                                    9             151      15              54   
                                                    10            102      2               37   

                                                                                    RFID  \
Subject Experiment Trainset                Parcours Parcours_Step Mutation Gesture         
Alfred  2          _TRAINSET14022017144824 101      1             151      15        142   
                                                    2             101      1          72   
                                                    3             152      17        102   
                                                    4             101      1          57   
                                                    5             152      17         77   
                                                    6             101      1          49   
                                                    7             152      17         66   
                                                    8             101      1          46   
                                                    9             152      17         65   
                                                    10            101      1          44   
                   _TRAINSET14022017144923 102      1             151      15         88   
                                                    2             102      2          54   
                                                    3             153      17         74   
                                                    4             102      2          37   
                                                    5             153      17         61   
                                                    6             102      2          41   
                                                    7             153      17         50   
                                                    8             102      2          35   
                                                    9             151      15         54   
                 

In [53]:
# Therefore Group by PARCOURS
# group data nicely, subdivided by Subject, Experiment, Trainset, Parcours
group_by = ('Subject','Experiment','Trainset','Parcours')

In [54]:
# Step 1: replace ALL Nestures by NaN
if replace_nestures:
    # make a copy of the complete data before altering anything
    data_nonest = data.copy()
    idx_nestures = data_nonest['Gesture'].isin(nestures)
    # replace nestures by NaN
    data_nonest.loc[idx_nestures,'Gesture'] = np.nan
    print data_nonest.head()

In [55]:
# now we can use the Forward FILL and Backward FILL methods of Pandas
# to replace the NaNs by the values that come before or after

# BUT: we shall not do that across Parcours/Experiments!
# GROUPBY helps us here to apply the fill methods only within a PARCOURS

if replace_nestures:
    # BACKWARD FILL first by later values to NaNs before
    data_nonest = data_nonest.groupby(group_by).bfill()

    # in case there would be NaNs left, do also a FORWARD FILL
    #data = data.groupby(group_by).ffill()
    
    print "Replaced Nestures by filling with neighboured Gestures!"
    print np.isnan(data_nonest['Gesture']).sum(), "NaN values remaining. Should be 0."
    # NOTE: bfill applies to ALL COLUMNS! so there might be other columns affected by this!
    # TODO double-check any side effects!
    
    # adding NaNs cause the Gesture column to be converted from int to float
    # we convert back to int
    data_nonest['Gesture'] = data_nonest['Gesture'].astype(int)
    
    # check/verify via groupby:
    group_by = ('Subject','Experiment','Trainset','Parcours','Gesture')
    group_df = data_nonest.groupby(group_by)
    print "After nesture replacement", len(group_df), "individual gesture blocks"

In [56]:
#group_df.size()

In [57]:
# from here on we use data again for data_nonest

if replace_nestures:
    # keep original data in a variable
    data_orig = data
    data = data_nonest

## Data Pre-Procssing Part I

### Which Sensor Parameters to use?

In [58]:
include_GRASP = True

if include_GRASP:
    params = ['AX', 'AY', 'AZ', 'EX', 'EY', 'EZ', 'GRASP_A', 'GRASP_B', 'GRASP_C']
else:
    params = ['AX', 'AY', 'AZ', 'EX', 'EY', 'EZ']

# TODO add RFID?

### Global Normalize?

#### Normalize Parameter columns to -1, 1

here it's done globally. if set to False, there is an option to do it locally later

In [59]:
normalize_global = True
# normalize_global means we normalize all parameter columns at once, globally => NO LATER TREATMENT

In [60]:
data[params].head()

,AX,AY,AZ,EX,EY,EZ,GRASP_A,GRASP_B,GRASP_C
0,0.06,-0.02,-0.10,216.8125,9.0625,-81.9375,781,8,797
1,0.09,-0.04,-0.11,217.0625,9.0625,-81.9375,782,0,799
2,0.12,-0.09,0.09,217.4375,9.1250,-81.8750,782,6,798
3,0.08,-0.08,0.03,217.6250,9.1250,-81.8125,784,7,798
4,0.07,-0.09,0.04,217.9375,9.1875,-81.7500,781,0,798


In [61]:
if normalize_global:
    # normalize to -1, 1
    data[params] = preprocessing.minmax_scale(data[params], feature_range=(-1, 1), axis=0, copy=False)

In [62]:
data[params].head()

,AX,AY,AZ,EX,EY,EZ,GRASP_A,GRASP_B,GRASP_C
0,0.067858,-0.035166,0.170465,0.204723,-0.030690,-0.455366,0.469298,-0.981132,0.639229
1,0.068614,-0.035848,0.170169,0.206112,-0.030690,-0.455366,0.473684,-1.000000,0.646235
2,0.069369,-0.037555,0.176088,0.208196,-0.029892,-0.455019,0.473684,-0.985849,0.642732
3,0.068362,-0.037214,0.174312,0.209238,-0.029892,-0.454672,0.482456,-0.983491,0.642732
4,0.068110,-0.037555,0.174608,0.210974,-0.029095,-0.454324,0.469298,-1.000000,0.642732


## Get Isolated Gestures

### Grouping for each Gesture:

### Select correct level of detail:
* a) Parcours: all same gestures of a Parcours will be concatenated together
* b) Parcours-Step: gestures inside one Parcours will be keep individual

In [63]:
# select: 'Parcours' or 'Parcours_Step':
#level_of_detail = 'Parcours'
level_of_detail = 'Parcours-Step' 

In [64]:
# GET INDIVIDUAL GESTURES 
# group data nicely, subdivided by Subject, Experiment, Trainset, Parcours or Parcours-Step, Gesture

if level_of_detail == 'Parcours':
    group_by = ('Subject','Experiment','Trainset','Parcours','Gesture')
elif level_of_detail == 'Parcours-Step':
    group_by = ('Subject','Experiment','Trainset','Parcours','Parcours_Step','Gesture')
else:
    raise ValueError("invalid level_of_detail")
    
group_df = data.groupby(group_by)
group_df.mean().head(100)  # mean is not meaningful here as aggregation - just to print the structure of the data

TimeStamp  \
Subject Experiment Trainset                Parcours Parcours_Step Gesture              
Alfred  2          _TRAINSET14022017144824 101      1             15         1676972   
                                                    2             1          4500061   
                                                    3             17         6842075   
                                                    4             1          9005396   
                                                    5             17        10878720   
                                                    6             1         12653858   
                                                    7             17        14322486   
                                                    8             1         15933183   
                                                    9             17        17529615   
                                                    10            1         19103138   
                   _TRAINSET14022017144923 102      1             15         1033315   
                                                    2             2          3023165   
                                                    3             17         4844222   
                                                    4             2          6444991   
                                                    5             17         7890484   
                                                    6             2          9379091   
                                                    7             17        10744372   
                                                    8             2         12008516   
                                                    9             15        13349230   
                                                    10            2         14761164   
                   _TRAINSET14022017145122 103      1             15         1809063   
                                                    2             1          4899894   
                                                    3             2          6778309   
                                                    4             1          8480780   
                                                    5             2         10135314   
                                                    6             1         11708673   
                                                    7             2         13423902   
                                                    8             1         15154814   
                                                    9             2         16920531   
                                                    10            1         18651971   
...                                                                              ...   
                   _TRAINSET14022017145629 109      9             2         12399662   
                                                    10            1         13631723   
                                                    11            2         14896240   
                   _TRAINSET14022017145751 110      1             15         1209033   
                                                    2             2          3408631   
                                                    3             1          5215754   
                                                    4             2          7174987   
                                                    5             1          9163900   
                                                    6             2         11071770   
                                                    7             1         12896470   
                                                    8             2         14637548   
                                                    9             1         16261553   
                                                    10            2         17646682   
                           

In [65]:
print len(group_df), "individual gesture blocks"

5169 individual gesture blocks


## Get Gesture Data: 1 Block per each individual Gesture

we put each time series that belong to 1 particular gesture in a particular parcours into a dictionary,
which contains a list of such time series blocks per gesture entry in the dict

### Create Gesture Dictionary + Reduce Data to desired parameter columns

In [66]:
# now we ITERATE nicely through group_df and get each Gesture block individually
# -> group_data will be a dataframe just for a single gesture

i=0
# dictionary containing a list of sub-datasets for each gesture, to train ML
gesture_exp_dict = {}

for name_tuple, datablock in group_df:
    i += 1
    #print str(name_tuple)
    gesture = name_tuple[-1]  # gesture is last element of tuple, as defined in group_by above
    
    # initalize empty list for the gesture if not existing in dict yet
    if gesture not in gesture_exp_dict.keys():
        gesture_exp_dict[gesture] = [] 
        
    # reduce dataframe to params columns
    datablock = datablock[params] 
    
    # add data to gesture dict
    gesture_exp_dict[gesture].append(datablock)
    
    # NOTE that group_data here still contains ALL data columns. we will redue to params later

print "DONE:", i, "gesture blocks"

DONE: 5169 gesture blocks


#### Some Statistics about the Gesture Block data

In [67]:
# How many data blocks = training examples do we have for each gesture
for gest in sorted(gesture_exp_dict.keys()):
    print "G", gest, '\t', len(gesture_exp_dict[gest]), "training data blocks", '\t', gesture_name(gest) 

G 1 	600 training data blocks 	Single Rotation klein rechtsrum
G 2 	600 training data blocks 	Single Rotation klein linksrum
G 3 	325 training data blocks 	Oszillierende Rotation klein rechtsrum
G 4 	325 training data blocks 	Oszillierende Rotation klein linksrum
G 5 	300 training data blocks 	Single Rotation groß rechtsrum
G 6 	300 training data blocks 	Single Rotation groß linksrum
G 7 	100 training data blocks 	Oszillierende Rotation groß rechtsrum
G 8 	100 training data blocks 	Oszillierende Rotation groß linksrum
G 9 	20 training data blocks 	Kontinuierliche Rotation groß rechtsrum
G 10 	20 training data blocks 	Kontinuierliche Rotation groß linksrum
G 11 	520 training data blocks 	LinearMovement Single
G 12 	160 training data blocks 	LinearMovement Oszillierend
G 13 	60 training data blocks 	Drücken
G 14 	60 training data blocks 	Rest
G 15 	511 training data blocks 	Greifen
G 16 	516 training data blocks 	Positionieren
G 17 	652 training data blocks 	Umgreifen


In [68]:
# how many data points (= samples or timesteps) does each data block have?

data_sizes = {} # collect per gesture in dict
data_sizes_total = [] # collect all in list

print "Average length per gesture:"

for gest in sorted(gesture_exp_dict.keys()):
    print "G", gest, ':\t', 
    data_sizes[gest] = []
    for datablock in gesture_exp_dict[gest]:
        size = datablock.shape[0]
        data_sizes[gest].append(size)
        data_sizes_total.append(size)
    avg_gesture_len = int(np.mean(data_sizes[gest]))
    print "% d samples \t%0.2f sec" % (avg_gesture_len, avg_gesture_len * time_delta_sec)

Average length per gesture:
G 1 :	 61 samples 	1.46 sec
G 2 :	 59 samples 	1.42 sec
G 3 :	 296 samples 	7.11 sec
G 4 :	 311 samples 	7.47 sec
G 5 :	 70 samples 	1.68 sec
G 6 :	 68 samples 	1.63 sec
G 7 :	 520 samples 	12.48 sec
G 8 :	 482 samples 	11.57 sec
G 9 :	 356 samples 	8.55 sec
G 10 :	 376 samples 	9.03 sec
G 11 :	 82 samples 	1.97 sec
G 12 :	 339 samples 	8.14 sec
G 13 :	 65 samples 	1.56 sec
G 14 :	 93 samples 	2.23 sec
G 15 :	 134 samples 	3.22 sec
G 16 :	 114 samples 	2.74 sec
G 17 :	 77 samples 	1.85 sec


In [69]:
# minimum, average and maximum sample size of gestures
gest_len_min = min(data_sizes_total)
gest_len_avg = np.mean(data_sizes_total)
gest_len_max = max(data_sizes_total)

In [70]:
gest_len_min, gest_len_avg, gest_len_max

(19, 137.62042948345908, 1038)

In [71]:
# average data length (number of samples)
print "Average length all gestures"
print "% d samples \t%0.2f sec" % (gest_len_avg, gest_len_avg * time_delta_sec)    

Average length all gestures
 137 samples 	3.30 sec


In [72]:
pd.Series(data_sizes_total).describe()

count    5169.000000
mean      137.620429
std       131.439739
min        19.000000
25%        56.000000
50%        84.000000
75%       159.000000
max      1038.000000
dtype: float64

## Pre-Processing

### Low-Pass Filter

removing high frequencies (little fluctuations which are probably not relevant)

In [73]:
# source code from https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units

from scipy.signal import butter, lfilter, freqz

def butter_lowpass(cutoff, fs, order=5):
    '''cutoff: cutoff frequency in Hz
    fs: sampling rate in Hz'''
    nyq = 0.5 * fs # Nyquist frequency is half the sampling rate (fs)
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

### Set Filter Settings here

In [74]:
# Filter settings (global vars)

# filter order (see scipy: butter)
order = 1 #3 #5 #6

# sampling rate in Hz
#fs = 30.0       # fixed assumed sample rate
fs = sampling_rate   # determined before by average time delta 

# cutoff frequency of the filter in Hz
cutoff = 4 #Hz
#cutoff = 3.667 
#cutoff = 1.3
#cutoff = 0.667 
#cutoff = 0.5
#cutoff = 0.33

### Preprocessing Function

In [75]:
def preprocess_signal(testdata, 
                      normalize=False, 
                      resampling=False, n_samples=None, timestamps=None, window='hann', 
                      filtering=False):
    
    # Min/max normalization
    # Note: to do it the fully right way, the minmax scaling should be done on all training data coherently
    # (currently its done per training block) and the same scaling values (min and max) should be reused here
    # see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
    if normalize:
        testdata = preprocessing.minmax_scale(testdata, feature_range=(-1, 1), axis=0)
        
    # Time Resampling
    if resampling:
        
        if n_samples is None:
            # if not a FIXED number of samples is provided, the number of samples stays the same as in the input signal
            n_samples = testdata.shape[0] 
        
        if timestamps is None:
            # NO timestamp alignment, just resampling to a given number of target samples
            testdata = resample(testdata, num=n_samples, window=window)
        else:
            # if provided, we use the original timestamps to re-align the signal
            # TODO check: length of signals in testdata must match len(timestamps)
            testdata, timestamps2 = resample(testdata, num=n_samples, t=timestamps, window='hann')
        

    if filtering:
        # filter the signal block with low-pass filter
        testdata = butter_lowpass_filter(testdata, cutoff, fs, order)
        
    return testdata

## Feature Calculation

### Zero Crossing Rate

In [76]:
def calc_zero_crossings(datablock, normalized=False):
    '''computes row-wise zerocrossings'''
    # datablock is assumed to be pandas Dataframe and to have multiple signals in the rows
    # example for 1 signal row:
    #zcr = np.signbit(signal).diff().abs().mean()
    # for multiple signal rows:
    zcr = np.signbit(datablock).astype(int).diff(axis=0).abs().mean(axis=0)
    
    if normalized:
        # divide by length of signal, otherwise it will be directly related to the size of the chosen window
        zcr = zcr / datablock.shape[0]
    return zcr

### Statistical Features

In [77]:
# Calc statistical features

def calc_statistical_features(matrix, axis=0):

    # to define the proper output shape, we need the "other axis" of the input shape (not the one where we compute along)
    other_axis = int(not axis) 
    n_rows = matrix.shape[other_axis]
    
    result = np.zeros((n_rows,7))
    
    result[:,0] = np.mean(matrix, axis=axis)
    result[:,1] = np.var(matrix, axis=axis, dtype=np.float64) 
    result[:,2] = stats.skew(matrix, axis=axis)
    result[:,3] = np.median(matrix, axis=axis)
    result[:,4] = np.min(matrix, axis=axis)
    result[:,5] = np.max(matrix, axis=axis)
    result[:,6] = stats.kurtosis(matrix, axis=axis, fisher=False) # Matlab calculates Pearson's Kurtosis

    result[np.where(np.isnan(result))] = 0
    return result

### Power Spectrum (~ FFT)

In [78]:
# TODO: set window size to something like 1/2 or 1/3 of the smallest gesture

def compute_power_spectrum_avg(signal_data, window_size=100):   # window_size=360
    '''signal_data: numpy array, rows: time, columns: different signals
    computes power spectrum of each signal, in a windowed manner
    aftewards the power spectrums of each window per signal are averaged'''
    
    if window_size > len(signal_data):
        raise ValueError("Window is bigger than input signal: " + str(len(signal_data)))
    
    hop_size = int(window_size / 2)

    power_spec_list = []

    pos = 0
    while pos + window_size <= len(signal_data):
        #print pos, pos+window_size
        sig_window = signal_data[pos:pos+window_size]
        # for periodogram we need to transpose to compute the signal along the right axis
        freq_axis, power_spec = periodogram(sig_window.T, fs=sampling_rate, window='hann') 
        power_spec_list.append(power_spec)
        pos += hop_size

    power_spec_array = np.array(power_spec_list)  
    power_spec_avg = np.mean(power_spec_array, axis=0)
    return power_spec_avg, freq_axis

### Function to compute All features

In [79]:
def calc_all_features(in_data, calc_derivative=False, calc_zerocrossings=False, calc_power_spectrum=False):

    # calc statistical features
    features = calc_statistical_features(in_data, axis=0)

    # vectorize
    features = features.flatten()

    if calc_derivative:
        # calc derivative of all signals
        in_data_deriv = np.gradient(in_data, axis=0)
        # calc statistics of derivatives
        features_deriv = calc_statistical_features(in_data_deriv, axis=0)
        # vectorize
        features_deriv = features_deriv.flatten()
        # concatenate to other features
        features = np.concatenate((features,features_deriv))

    if calc_zerocrossings:
        features_zcr = calc_zero_crossings(in_data)
        features = np.concatenate((features,features_zcr))
        
    if calc_power_spectrum:
        power_spec_avg, freq_axis = compute_power_spectrum_avg(in_data)
        # combine spectrum of all signals into 1 long vector
        power_spec_avg = power_spec_avg.flatten()
        features = np.concatenate((features,power_spec_avg))

    return features

## Start Feature Calculation

### Set Options here:

In [80]:
# OPTIONS:

# want non_gestures in result? can use option depending on replace_nestures True or False, or exclude always
exclude_non_gestures = replace_nestures  # True

# preprocessing options
use_lowpassfilter = False     # apply low-pass filter (filtering out high frequencies) - settings see above
use_normalized = True         # local normalization, per each window!
use_resampled = True          # resample time signal to common window length (see samples parameter)
# 2 choices for resampling:
# a) just re-align by given timestamps to equi-distant timestamps, but same number of samples are kept:
n_samples = None
# b) resample ALL gestures to a FIXED common number of samples given here:
#n_samples = int(gest_len_avg)   # we choose the average gesture length for the common resampled gesture length

# override use_normalized: if we already normalized globally, we should not do it again locally
if normalize_global: use_normalized = False

# feature options: # True is better for all, except Power spectrum
calc_derivative = True
calc_zerocrossings = True
calc_powerspectrum = False

In [81]:
# we added preprocess_signal() function below, thats why we need to use the original gesture_dict as input
input_dict = gesture_exp_dict 

In [82]:
if exclude_non_gestures:
    gestures_to_process = gestures_pos
else:
    gestures_to_process = input_dict.keys()

In [83]:
gestures_pos

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [84]:
# COMPUTE FEATURES
# LOOP over all gesture data to create features

# initialize feature output for training data as a list
train_list = []
train_classes_num = []

# iterate over gesture number
for gest in sorted(gestures_to_process):
    print "G", gest, ':\t', len(input_dict[gest]), "examples"
    
    # iterate over all gesture blocks for that gesture number
    for in_data in input_dict[gest]:
        #print datablock.shape, 
        
        #if use_resampled:
        #    # resampled data has already extracted the param columns
        #    in_data = datablock
        #else:
        #    # for non-resampled we have to get the relevant data columns and transpose
        #    in_data = datablock[params].T
        
        # preprocessing
        in_data = preprocess_signal(in_data, use_normalized, 
                                    use_resampled, n_samples, timestamps=None, window=None, # 'hann'
                                    filtering=use_lowpassfilter)
                
        # convert to dataframe cause we use pandas .diff() in ZCR computation
        in_data = pd.DataFrame(in_data, columns=params)

        # calculate features
        features = calc_all_features(in_data, calc_derivative, calc_zerocrossings, calc_powerspectrum)

        # append to output list
        train_list.append(features)
        
        # store class (gesture number) for these features
        train_classes_num.append(gest)

G 1 :	600 examples
G 2 :	600 examples
G 3 :	325 examples
G 4 :	325 examples
G 5 :	300 examples
G 6 :	300 examples
G 7 :	100 examples
G 8 :	100 examples
G 9 :	20 examples
G 10 :	20 examples
G 11 :	520 examples
G 12 :	160 examples
G 13 :	60 examples
G 14 :	60 examples
G 15 :	511 examples
G 16 :	516 examples
G 17 :	652 examples


In [85]:
print "Feature vector length:", len(features)

Feature vector length: 135


# Machine Learning

### Prepare Training Data

In [86]:
print "Training data:", len(train_list), "examples"

Training data: 5169 examples


In [87]:
# make feature array from feature list (ALL training data)
train_data = np.array(train_list)
del train_list # not needed any longer
train_data.shape

(5169, 135)

In [88]:
# verify if the training categories (gesture numbers) have the same length
len(train_classes_num)

5169

### Standardize

Zero-mean unit-variance Standardization

In [89]:
# ad-hoc scaling
# train_data = preprocessing.scale(train_data,axis=0)
# axis=0 means independently standardize each feature, otherwise (if 1) standardize each sample

In [90]:
# we now user StandardScaler class to keep the mean and variance for later
standardizer = preprocessing.StandardScaler()
train_data = standardizer.fit_transform(train_data)

### Train/Test Set Split

In [91]:
# split the data into train/test set

testset_size = 0.25

# sklearn >= 0.18
# use random_state to avoid that the results fluctuate randomly
splitter = StratifiedShuffleSplit(n_splits=1, test_size=testset_size, random_state=0) 
splits = splitter.split(train_data, train_classes_num)

# Note: this for loop is only executed once, if n_splits==1
for train_index, test_index in splits:
    #print "TRAIN INDEX:", train_index
    #print "TEST INDEX:", test_index
    
    # split the data
    train_set = train_data[train_index]
    test_set = train_data[test_index]
    
    # and the numeric classes (groundtruth)
    train_classes = np.array(train_classes_num)[train_index]
    test_classes = np.array(train_classes_num)[test_index]
    
    print "TRAIN SIZE:", train_set.shape
    print "TEST SIZE:", test_set.shape
    

TRAIN SIZE: (3876, 135)
TEST SIZE: (1293, 135)


## 1) Gesture Regonition - isolated

### ML Algorithm: SVM

Support Vector Machines

In [155]:
# try 3 different SVM kernels
kernels = ['linear','poly','rbf']

In [156]:
models = {}

for kernel in kernels:
    print "SVM", kernel,
    
    # TRAIN 
    start_time = time.time() # measure time

    model = OneVsRestClassifier(SVC(kernel=kernel)) #, degree=degree)) #, n_jobs=-1)  # n_jobs = n cpus, -1 = all
    # full set
    #model.fit(train_data, train_classes_num)
    # train set
    model.fit(train_set, train_classes)
    
    # store in dict
    models[kernel] = model

    end_time = time.time()
    print "Training time:", timestr(end_time - start_time)

SVM linear Training time: 0:00:08
SVM poly Training time: 0:00:06
SVM rbf Training time: 0:00:06


#### Verification on Train Set (just for plausibility)

In [157]:
# predict on train set (will reuse last model == rbf only)
pred_train = model.predict(train_set)
# print pred_train

In [158]:
#print train_classes

In [159]:
# Accuracy on train set (manual computation)
np.sum(pred_train == train_classes) * 1.0 / len(train_classes)

0.94969040247678016

In [160]:
# Accuracy on train set (using scikit-learn)
accuracy_score(train_classes, pred_train)

0.94969040247678016

## Evaluation

### Evaluation - Overall

In [161]:
result_ov = pd.DataFrame(index=kernels, columns=['Accuracy','Precision','Recall','F-Measure'])

In [162]:
for k in kernels:
    # predict on TEST set
    pred_test = models[k].predict(test_set) 
    
    # Accuracy, Precision, Reacall on TEST set
    result_ov.loc[k,'Accuracy'] = accuracy_score(test_classes, pred_test)
    result_ov.loc[k,'Precision'] = precision_score(test_classes, pred_test, average='macro')
    result_ov.loc[k,'Recall'] = recall_score(test_classes, pred_test, average='macro')
    result_ov.loc[k,'F-Measure'] = f1_score(test_classes, pred_test, average='macro')

In [163]:
pd.options.display.float_format = '{:,.2f}'.format
result_ov = result_ov*100
result_ov

,Accuracy,Precision,Recall,F-Measure
linear,81.44,78.60,82.06,79.98
poly,84.61,85.70,81.85,82.48
rbf,86.31,87.13,85.23,85.62


In [165]:
# select the best model
crit = 'F-Measure'  # select best model by which criteria
best_k = np.argmax(result_ov[crit])
print "Best model type is:", best_k
best_model = models[best_k]

Best model type is: rbf


### Evaluation - Per Gesture

In [166]:
# Predict again with best model
pred_test = best_model.predict(test_set) 

In [167]:
# TODO check if the sorting of precision_score etc. is really in this order!!
labels = sorted(np.unique(test_classes))
gesture_names = [gesture_name(l) for l in labels]

In [168]:
# nice result dataframe
columns = ['Gesture','N_train','N_test','Precision','Recall','F1']
result_df = pd.DataFrame(index=labels,columns=columns)
result_df['Gesture'] = gesture_names

In [169]:
# number of train / test instances
values, counts = np.unique(train_classes, return_counts=True)
result_df['N_train'] = pd.Series(counts, index=values)
values, counts = np.unique(test_classes, return_counts=True)
result_df['N_test'] = pd.Series(counts, index=values)

In [170]:
# per class evaluation
result_df['Precision'] = precision_score(test_classes, pred_test, average=None) * 100
result_df['Recall'] = recall_score(test_classes, pred_test, average=None) * 100
result_df['F1'] = f1_score(test_classes, pred_test, average=None) * 100

In [171]:
result_df

,Gesture,N_train,N_test,Precision,Recall,F1
1,Single Rotation klein rechtsrum,450,150,87.04,94.00,90.38
2,Single Rotation klein linksrum,450,150,85.98,94.00,89.81
3,Oszillierende Rotation klein rechtsrum,244,81,94.44,83.95,88.89
4,Oszillierende Rotation klein linksrum,243,82,89.53,93.90,91.67
5,Single Rotation groß rechtsrum,225,75,94.81,97.33,96.05
6,Single Rotation groß linksrum,225,75,89.02,97.33,92.99
7,Oszillierende Rotation groß rechtsrum,75,25,82.14,92.00,86.79
8,Oszillierende Rotation groß linksrum,75,25,90.91,80.00,85.11
9,Kontinuierliche Rotation groß rechtsrum,15,5,83.33,100.00,90.91
10,Kontinuierliche Rotation groß linksrum,15,5,100.00,80.00,88.89


In [173]:
# compare average P, R and F to overall P, R and F from small evaluation table above (overal result is the same)
result_df.mean(axis=0).to_frame().T

,N_train,N_test,Precision,Recall,F1
0,228.00,76.06,87.13,85.23,85.62


In [108]:
# Confusion Matrix
conf = confusion_matrix(test_classes, pred_test, labels=labels) # labels defines the order
labels_long = gestures_df.loc[labels,'name']
conf_df = pd.DataFrame(conf, index=labels_long, columns=labels)
conf_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
name,,,,,,,,,,,,,,,,,
Single Rotation klein rechtsrum,136,3,0,0,1,1,0,0,0,0,2,0,0,0,0,3,4
Single Rotation klein linksrum,1,140,0,0,0,0,0,0,0,0,2,0,1,1,1,0,4
Oszillierende Rotation klein rechtsrum,0,3,70,5,0,0,0,0,0,0,0,0,0,0,0,3,0
Oszillierende Rotation klein linksrum,0,1,0,78,0,0,0,0,0,0,1,0,0,0,0,2,0
Single Rotation groß rechtsrum,0,0,0,0,75,0,0,0,0,0,0,0,0,0,0,0,0
Single Rotation groß linksrum,0,0,0,0,0,74,0,0,0,0,0,0,0,0,0,1,0
Oszillierende Rotation groß rechtsrum,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0
Oszillierende Rotation groß linksrum,0,0,0,0,0,0,3,19,0,0,0,0,0,0,1,1,1
Kontinuierliche Rotation groß rechtsrum,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0


## 2) Continuous Time Series Prediction

What is our input stream?

The data of 1 trainset, because after each trainset, the TimeStamp is reset.

In [109]:
# a) loop over each Trainset
#group_by = ('Subject','Experiment','Trainset')

# b) use Experiment as the block where we do predictions (means it includes timestamp resets!!)
group_by = ('Subject','Experiment')

group_df = data.groupby(group_by)
group_df.max().head(50) 

Trainset  TimeStamp          RFID  GRASP_A  \
Subject Experiment                                                              
Alfred  2           _TRAINSET14022017172235   44132577  760057911BAB     0.90   
        8           _TRAINSET18022017151654  122720319  280015E55981     0.82   
Andreas 1           _TRAINSET14022017123316   48620325  760057911BAB     1.00   
        6           _TRAINSET17022017155347  152209477  09006734114B     0.97   
Claudia 3           _TRAINSET15022017120738   71354787  760057911BAB     0.79   
        5           _TRAINSET17022017115327  170698407  280015E55981     0.78   
Dominik 4           _TRAINSET15022017173325   68968234  760057911BAB     0.88   
        7           _TRAINSET18022017114128  147807390  09006734114B     0.91   

                    GRASP_B  GRASP_C   AX   AY   AZ   EX   EY   EZ  Parcours  \
Subject Experiment                                                             
Alfred  2              0.95     0.89 0.47 0.32 0.61 1.00 1.00 1.00       802   
        8              0.96     0.85 0.81 0.58 0.81 1.00 0.98 0.88       903   
Andreas 1              0.95     1.00 1.00 1.00 0.60 1.00 1.00 1.00       802   
        6              1.00     0.99 0.84 0.84 0.76 1.00 1.00 1.00       903   
Claudia 3              0.79     0.81 0.72 0.74 0.57 1.00 1.00 1.00       802   
        5              0.80     0.78 0.84 0.67 1.00 1.00 1.00 1.00       903   
Dominik 4              0.85     0.92 0.46 0.63 0.78 1.00 1.00 1.00       802   
        7              0.96     0.93 0.93 0.70 0.84 1.00 1.00 1.00       903   

                    Parcours_Step  Mutation  Host Host/Spot  Gesture  
Subject Experiment                                                    
Alfred  2                      12       853   127         B       17  
        8                      12       956   122         A       16  
Andreas 1                      12       853   122         B       17  
        6                      12       956   127         A       17  
Claudia 3                      12       853   127         B       17  
        5                      12       956   122         A       16  
Dominik 4                      12       853   127         B       17  
        7                      12       956   122         A       16

In [110]:
print len(group_df), "Experiments / Trainsets"

8 Experiments / Trainsets


In [111]:
# iterate over each Trainset
i =0
for name_tuple, group_data in group_df:
    i += 1
    #print str(name_tuple)
    
    if len(name_tuple) == 3:
        subject, exp, trainset = name_tuple
    elif len(name_tuple) == 2:
        subject, exp = name_tuple
        trainset = None
    
    break # for testing we just do 1 loop
    

In [112]:
name_tuple

('Alfred', 2)

In [113]:
group_data['TimeStamp'].min()

0

In [114]:
group_data['TimeStamp'].max()

44132577

In [115]:
if len(name_tuple) == 3:
    # check if TimeStamps are monotonously increasing
    if not np.all(group_data['TimeStamp'].diff()[1:] > 0):
        raise ValueError("Time Stamps are not monotonously increasing!")

In [116]:
# set these to None so that plot title is not shown wrongly
parcours = None
mutation = None
gesture = None

In [117]:
# which gestures appear in this Experiment or Trainset
group_data['Gesture'].unique()

array([15,  1, 17,  2,  6,  5, 16, 11, 10,  9])

### Pre-Process the Data - Testing

the same way as it was done for training set

In [118]:
pd.options.display.float_format = '{:,.5f}'.format

In [119]:
# get the relevant columns out of group_data

In [120]:
timestamps = group_data['TimeStamp'].tolist()

In [121]:
test_gestures = group_data['Gesture'].tolist()

In [122]:
# 9 parameters columns
testdata = group_data[params]
testdata.shape

(64432, 9)

In [123]:
# Global Min/max normalization
# Note: to do it the fully right way, the minmax scaling should be done on all training data coherently
# (currently its done per training block) and the same scaling values (min and max) should be reused here
# see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
# TODO store minmax_scale from training data and reapply same scaling here

if normalize_global:
    testdata = preprocessing.minmax_scale(testdata, feature_range=(-1, 1), axis=0, copy=False)

In [124]:
# convert to dataframe cause plot needs column names
testdata = pd.DataFrame(testdata, columns=params)
testdata.head(15)

,AX,AY,AZ,EX,EY,EZ,GRASP_A,GRASP_B,GRASP_C
0,0.13834,-0.22791,-0.15303,0.27418,-0.05399,-0.44762,-0.02110,0.16147,0.11688
1,0.13615,-0.22791,-0.14990,0.27418,-0.05399,-0.44762,-0.02954,0.15581,0.11688
2,0.13288,-0.23140,-0.15773,0.27383,-0.05399,-0.44762,-0.02954,0.15014,0.10823
3,0.12960,-0.23721,-0.15773,0.27383,-0.05399,-0.44762,-0.02954,0.15581,0.10823
4,0.12906,-0.23372,-0.16634,0.27383,-0.05399,-0.44762,-0.02110,0.15581,0.12554
5,0.13233,-0.22442,-0.15930,0.27383,-0.05399,-0.44762,-0.02954,0.15014,0.09957
6,0.13452,-0.22907,-0.16243,0.27383,-0.05399,-0.44762,-0.03797,0.15014,0.12554
7,0.13615,-0.22558,-0.15303,0.27418,-0.05399,-0.44762,-0.02110,0.15581,0.13420
8,0.14379,-0.22558,-0.15382,0.27453,-0.05399,-0.44796,-0.02110,0.15581,0.11688
9,0.14488,-0.23140,-0.14521,0.27453,-0.05482,-0.44796,-0.02954,0.15014,0.11688


In [125]:
# time resample

n_samples = len(timestamps)  

if use_resampled:
    # the number of samples stays the same
    # but we use the original timestamps to re-align the signal
    testdata_res, timestamps2 = resample(testdata, num=n_samples, t=timestamps)
    
    # convert to dataframe cause plot needs column names
    testdata_res = pd.DataFrame(testdata_res, columns=params)

In [126]:
timestamps[:15]

[0,
 24915,
 49032,
 74016,
 111220,
 122772,
 143717,
 172480,
 186423,
 211319,
 237536,
 261141,
 302690,
 305197,
 341003]

In [127]:
timestamps2[:15]

array([      0.,   24915.,   49830.,   74745.,   99660.,  124575.,
        149490.,  174405.,  199320.,  224235.,  249150.,  274065.,
        298980.,  323895.,  348810.])

In [128]:
# timestamps are now equidistant
timestamps2[1:15] - timestamps2[:14]

array([ 24915.,  24915.,  24915.,  24915.,  24915.,  24915.,  24915.,
        24915.,  24915.,  24915.,  24915.,  24915.,  24915.,  24915.])

In [129]:
testdata_res.head(15)

,AX,AY,AZ,EX,EY,EZ,GRASP_A,GRASP_B,GRASP_C
0,0.13834,-0.22791,-0.15303,0.27418,-0.05399,-0.44762,-0.02110,0.16147,0.11688
1,0.13615,-0.22791,-0.14990,0.27418,-0.05399,-0.44762,-0.02954,0.15581,0.11688
2,0.13288,-0.23140,-0.15773,0.27383,-0.05399,-0.44762,-0.02954,0.15014,0.10823
3,0.12960,-0.23721,-0.15773,0.27383,-0.05399,-0.44762,-0.02954,0.15581,0.10823
4,0.12906,-0.23372,-0.16634,0.27383,-0.05399,-0.44762,-0.02110,0.15581,0.12554
5,0.13233,-0.22442,-0.15930,0.27383,-0.05399,-0.44762,-0.02954,0.15014,0.09957
6,0.13452,-0.22907,-0.16243,0.27383,-0.05399,-0.44762,-0.03797,0.15014,0.12554
7,0.13615,-0.22558,-0.15303,0.27418,-0.05399,-0.44762,-0.02110,0.15581,0.13420
8,0.14379,-0.22558,-0.15382,0.27453,-0.05399,-0.44796,-0.02110,0.15581,0.11688
9,0.14488,-0.23140,-0.14521,0.27453,-0.05482,-0.44796,-0.02954,0.15014,0.11688


In [130]:
# debug check whether the values have been altered -> OK
#testdata == testdata_res

In [131]:
# overwrite testdata with testdata_res for subsequent coherent usage
#testdata = testdata_res

### Continuous Prediction

In [132]:
# for our window_size (= signal length of input to Machine Learning)
# we take the average signal length of the trained gestures
window_size = int(gest_len_avg) # gest_len_min 
window_size

137

In [133]:
# PREDICTION RESOLUTION
# how quickly do we step forward

# for now we choose half the window_size
#step_size = window_size / 2

# for speedup we choose same as window_size
step_size = window_size 

# can be set smaller for higher resolution

# TODO: set in milliseconds - convert back to sample length

step_size

137

In [134]:
print "Testing in continous signal with %.3f sec window and %.3f sec step size" % (window_size * time_delta_sec, 
                                                                                  step_size * time_delta_sec)

Testing in continous signal with 3.289 sec window and 3.289 sec step size


In [135]:
# TODO: align with preprocess_signal function used in training data above

def preprocess_signal_continuous(testdata, normalize=False, resampling=False, timestamps=None, filtering=False):
    
    # Min/max normalization
    # Note: to do it the fully right way, the minmax scaling should be done on all training data coherently
    # (currently its done per training block) and the same scaling values (min and max) should be reused here
    # see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

    if normalize:
        testdata = preprocessing.minmax_scale(testdata, feature_range=(-1, 1), axis=0)
        
    # Time Resampling
    if resampling:
        # the number of samples stays the same
        # if provided, we use the original timestamps to re-align the signal
        n_samples = testdata.shape[0] # must match len(timestamps)
        testdata, timestamps2 = resample(testdata, num=n_samples, t=timestamps) #, window='hann')

    if filtering:
        # filter the signal block with low-pass filter
        testdata = butter_lowpass_filter(testdata, cutoff, fs, order)
        
    return testdata

In [136]:
# PREDICTION LOOP OVER 1 TRAINING INPUT BLOCK

def continuous_prediction(testdata, test_gestures, window_size, step_size):
    pos = 0
    n_samples = testdata.shape[0]
    
    # output
    test_groundtruth = [] # we create the groundtruth to compare with here
    predictions = []  # predictions are collected here

    while pos < (n_samples - window_size):
        # 1) DETERMINE GROUNDTRUTH (for evaluation only, not used in predictions)
        
        # to get the "correct" gesture for that window, we cut the same part of the gesture information
        test_window_groundtruth = test_gestures[pos:pos+window_size]

        # HOW to determine groundtruth?
        # a) determine gesture from the window center
        gt_gesture = test_gestures[pos+(window_size/2)]
        
        # b) Majority vote: which is the predominant label in the whole window
        #gt_gesture = Counter(test_window_groundtruth).most_common()[0][0]

        # 2) DO PREDICTIONS
        
        # cut a window out of the incoming signal
        signal = testdata[pos:pos+window_size]

        # calc features
        features = calc_all_features(signal, calc_derivative, calc_zerocrossings)

        # reshape to row vector for standardize and predict below (= single input sample)
        features = features.reshape(1, -1)  
        
        # STANDARDIZE features, the same way as done in training (reusing those mean and var)
        features = standardizer.transform(features)

        # ML prediction of gesture
        pred_gesture = best_model.predict(features)[0]

        # add to groundtruth and prediction list
        test_groundtruth.append(gt_gesture)
        predictions.append(pred_gesture)

        # step forward
        pos += step_size
    
    return test_groundtruth, predictions

In [137]:
# LOOP over ALL Experiments or Trainsets

i = 0
n_groups = len(group_df)

test_groundtruth_all = [] # we create the groundtruth to compare with here
predictions_all = []  # predictions are collected here

for name_tuple, group_data in group_df:
    
    i += 1
    print "Experiment", i, "/", n_groups, ":", str(name_tuple), group_data.shape,
    
    # just metadata
    if len(name_tuple) == 3:
        subject, exp, trainset = name_tuple
    elif len(name_tuple) == 2:
        subject, exp = name_tuple
        trainset = None
    
    # get signals, timestamps and gesture groundtruth
    timestamps = group_data['TimeStamp'].tolist()   # timestamps
    testdata = group_data[params]                   # signals
    test_gestures = group_data['Gesture'].tolist()  # groundtruth
    
    # preprocess testdata
    print "Preprocessing ...",
    testdata = preprocess_signal_continuous(testdata, use_normalized, use_resampled, timestamps, use_lowpassfilter)
    #print testdata.shape
    
    # convert to dataframe cause we use pandas .diff() in ZCR computation
    testdata = pd.DataFrame(testdata, columns=params)
    
    print "Prediction:", 
    test_groundtruth, predictions = continuous_prediction(testdata, test_gestures, window_size, step_size)
    print len(predictions), "predictions"
    
    test_groundtruth_all.extend(test_groundtruth)
    predictions_all.extend(predictions)
    

Experiment 1 / 8 : ('Alfred', 2) (64432, 20) Preprocessing ... Prediction: 470 predictions
Experiment 2 / 8 : ('Alfred', 8) (85467, 20) Preprocessing ... Prediction: 623 predictions
Experiment 3 / 8 : ('Andreas', 1) (66713, 20) Preprocessing ... Prediction: 486 predictions
Experiment 4 / 8 : ('Andreas', 6) (118749, 20) Preprocessing ... Prediction: 866 predictions
Experiment 5 / 8 : ('Claudia', 3) (75340, 20) Preprocessing ... Prediction: 549 predictions
Experiment 6 / 8 : ('Claudia', 5) (107382, 20) Preprocessing ... Prediction: 783 predictions
Experiment 7 / 8 : ('Dominik', 4) (91863, 20) Preprocessing ... Prediction: 670 predictions
Experiment 8 / 8 : ('Dominik', 7) (101414, 20) Preprocessing ... Prediction: 740 predictions


In [138]:
print len(predictions_all), "predictions"

5187 predictions


In [139]:
print "collected true gestures include:"
np.unique(test_groundtruth_all).tolist()

collected true gestures include:


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [140]:
print "predicted gestures include:"
np.unique(predictions_all).tolist()

predicted gestures include:


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [154]:
#pd.DataFrame({'groundt':test_groundtruth_all, 'pred':predictions_all})

In [142]:
result_ov = pd.DataFrame(columns=['result']) #columns=['Accuracy','Precision','Recall','F-Measure'])

# Accuracy, Precision, Reacall on TEST set
result_ov.loc['Accuracy'] = accuracy_score(test_groundtruth_all, predictions_all)
result_ov.loc['Precision'] = precision_score(test_groundtruth_all, predictions_all, average='macro')
result_ov.loc['Recall'] = recall_score(test_groundtruth_all, predictions_all, average='macro')
result_ov.loc['F-Measure'] = f1_score(test_groundtruth_all, predictions_all, average='macro')
result_ov = result_ov * 100
result_ov

,result
Accuracy,56.75728
Precision,60.71559
Recall,50.53742
F-Measure,52.99497


#### Confusion Matrix

In [143]:
labels_long = gestures_df.loc[labels,'name']
conf = confusion_matrix(test_groundtruth_all, predictions_all, labels=labels) # labels defines the order
conf_df = pd.DataFrame(conf, index=labels_long, columns=labels)
conf_df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
name,,,,,,,,,,,,,,,,,
Single Rotation klein rechtsrum,77,92,27,5,2,0,2,2,0,0,2,0,0,0,16,15,46
Single Rotation klein linksrum,49,95,8,17,0,0,1,3,0,0,4,0,1,0,10,8,57
Oszillierende Rotation klein rechtsrum,8,17,547,68,0,0,0,2,0,0,9,0,1,1,6,21,25
Oszillierende Rotation klein linksrum,11,26,72,552,0,0,1,2,0,0,10,3,1,0,11,26,26
Single Rotation groß rechtsrum,2,4,0,0,60,21,17,0,0,0,1,0,0,0,4,35,12
Single Rotation groß linksrum,0,3,1,0,26,62,4,5,0,0,1,1,0,0,13,16,23
Oszillierende Rotation groß rechtsrum,0,1,0,1,16,15,289,5,0,0,1,0,0,0,10,29,12
Oszillierende Rotation groß linksrum,1,1,1,2,6,11,28,231,0,0,2,3,0,0,15,24,30
Kontinuierliche Rotation groß rechtsrum,0,0,0,0,0,0,0,0,27,0,0,0,0,0,9,17,0
